In [ ]:
!git clone https://github.com/nguyensyduc060299/SemanticWeb.git

Cloning into 'SemanticWeb'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 29 (delta 11), reused 25 (delta 10), pack-reused 0
Unpacking objects: 100% (29/29), done.


In [ ]:
%cd /content/SemanticWeb/Semantic-Web-Project-master
# !pip install rdflib

/content/SemanticWeb/Semantic-Web-Project-master


In [ ]:
import rdflib
import pandas as pd
g=rdflib.Graph()
result=g.parse("result_dbpedia.owl", "xml")
print("graph has %s statements.\n" % len(g))

graph has 10185 statements.



# 1. Query the movies were filmed in a specify **country**

In [ ]:
country = "United_States"
query0 = """
    PREFIX movie: <http://www.semanticweb.org/group3/ontologies/2022/8/untitled-ontology-21#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>  
    SELECT ?name ?country
    WHERE { ?film rdf:type movie:Movie .
            ?film movie:movieName ?name .
            ?film movie:filmedIn ?country .
            FILTER regex(str(?country), "%s")
        }
    """%country

results = g.query(query0)
print ('{0:40s} {1:40s}'.format("Name","Country"))
print ('{0:40s} {1:40s}'.format("------------------------------------","---------------------------------------"))
for name,country in results:
	print ('{0:40s} {1:40s}'.format(name,country))

Name                                     Country                                 
------------------------------------     --------------------------------------- 
Bomb the System                          http://dbpedia.org/resource/United_States
9/11: Press for Truth                    http://dbpedia.org/resource/United_States
8: The Mormon Proposition                http://dbpedia.org/resource/United_States
3:10 to Yuma                             http://dbpedia.org/resource/United_States
Bosko's Parlor Pranks                    http://dbpedia.org/resource/United_States
Bog                                      http://dbpedia.org/resource/United_States
Bowery to Broadway                       http://dbpedia.org/resource/United_States
Boogie Man: The Lee Atwater Story        http://dbpedia.org/resource/United_States
Boxing in Barrels                        http://dbpedia.org/resource/United_States


# 2. Query top 10 movies with the longest duration

In [ ]:
def process_duration(duration):
  duration = duration.toPython()
  if isinstance(duration, int):
    return duration
  else:
    duration = duration.n3()
    rs = int(float(duration.split("^^")[0].replace("\"","")))
    return rs

query1 = """
    PREFIX movie: <http://www.semanticweb.org/group3/ontologies/2022/8/untitled-ontology-21#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    SELECT ?name ?duration
    WHERE { ?film rdf:type movie:Movie .
            ?film movie:movieName ?name .
            ?film movie:durationSeconds ?duration .
    }
"""

rs = g.query(query1)

rs_pd = pd.DataFrame(rs, columns=['Movie','Duration'])
rs_pd['Duration'] = rs_pd['Duration'].apply(lambda x : process_duration(x))
rs_pd = rs_pd.sort_values(by=['Duration'], ascending= False)
rs_pd[0:10]

,Movie,Duration
67,3 Ninjas,21300
264,7G Rainbow Colony,11100
5,23 March 1931: Shaheed,11100
85,40+,10920
247,Mohra,10620
265,7G Rainbow Colony,10440
84,Mohana Sundaram,10260
236,3 Idiots,10260
62,7aum Arivu,10080
99,ABCD: American-Born Confused Desi,10020


# 3. Query top 10 actors who participated in the most movies

In [ ]:
query2 = """
    PREFIX movie: <http://www.semanticweb.org/group3/ontologies/2022/8/untitled-ontology-21#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    SELECT ?actor (COUNT(?name) as ?num_movie)
    WHERE { ?film rdf:type movie:Movie .
            ?film movie:movieName ?name .
            ?film movie:hasActor ?actor .
    }
    GROUP BY ?actor
    ORDER BY DESC(?num_movie)
    LIMIT 10
"""

rs = g.query(query2)

rs_pd = pd.DataFrame(rs, columns=['Name Actor','Number Movie'])
rs_pd

,Name Actor,Number Movie
0,http://dbpedia.org/resource/Naseeruddin_Shah,4
1,http://dbpedia.org/resource/Siddique_(actor),3
2,http://dbpedia.org/resource/Atul_Kulkarni,3
3,http://dbpedia.org/resource/Joseph_Gordon-Levitt,3
4,http://dbpedia.org/resource/Ben_Foster_(actor),3
5,http://dbpedia.org/resource/Dean_Martin,3
6,http://dbpedia.org/resource/Danielle_Darrieux,3
7,http://dbpedia.org/resource/Christian_De_Sica,3
8,http://dbpedia.org/resource/Victor_Wong_(actor...,3
9,http://dbpedia.org/resource/Dave_Franco,3


# 4.Top 10 movies with the most actors participated in

In [ ]:
query3 = """
    PREFIX movie: <http://www.semanticweb.org/group3/ontologies/2022/8/untitled-ontology-21#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    SELECT ?name (COUNT(?actor) as ?num_actor)
    WHERE { ?film rdf:type movie:Movie .
            ?film movie:movieName ?name .
            ?film movie:hasActor ?actor .
    }
    GROUP BY ?name
    ORDER BY DESC(?num_actor)
    LIMIT 10
"""

rs = g.query(query3)

rs_pd = pd.DataFrame(rs, columns=['Name Movie','Number Actor'])
rs_pd

,Name Movie,Number Actor
0,,24
1,7 Days to Vegas,18
2,47 Ronin,16
3,24/7 in Love,15
4,360,15
5,72 Tenants of Prosperity,14
6,4.3.2.1.,13
7,3:10 to Yuma,12
8,3 from Hell,12
9,Bosta,11


# 5.Query top 10 countries with the longest total length of movies

In [ ]:
query4 = """
    PREFIX movie: <http://www.semanticweb.org/group3/ontologies/2022/8/untitled-ontology-21#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    SELECT ?country ?duration
    WHERE { ?film rdf:type movie:Movie .
            ?film movie:durationSeconds ?duration .
            ?film movie:filmedIn ?country .
    }
"""

rs = g.query(query4)
rs_pd = pd.DataFrame(rs, columns=['Country','Duration'])
rs_pd['Duration'] = rs_pd['Duration'].apply(lambda x : process_duration(x))
rs_pd = rs_pd.groupby(['Country']).sum()
rs_pd = rs_pd.sort_values(by=['Duration'], ascending= False)
rs_pd[0:10]

,Duration
Country,
http://dbpedia.org/resource/India,160260
http://dbpedia.org/resource/Canada,41940
http://dbpedia.org/resource/United_States,37500
http://dbpedia.org/resource/France,35700
http://dbpedia.org/resource/Australia,29760
http://dbpedia.org/resource/Italy,23940
http://dbpedia.org/resource/Brazil,22200
http://dbpedia.org/resource/Norway,16920
http://dbpedia.org/resource/Nigeria,14220
